This notebook is part of the [Machine Learning class](https://github.com/erachelson/MLclass) by [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en).

License: CC-BY-SA-NC.

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Boosting</div>

Boosting is a procedure that combines several "weak" predictors into a powerful "committee". It belongs to the family of committee-based or ensemble methods in Machine Learning.

The most popular Boosting algorithm is AdaBoost.M1 (Freund & Schapire, 1997).

A motivation for Boosting:<br>
*AdaBoost with trees is the best off-the-shelf classifier in the world.* (Breiman 1998)<br>
It is not so true anymore today but still accurate enough in practice.

For recent criticism on that statement, see:<br>
**Random classification noise defeats all convex potential boosters.**<br>
P.H. Long, R.A. Servedio, *Machine Learning*, **78**(3), 287-304, (2008).

In this notebook we take a very practical approach. For a more thorough and rigorous presentation, see (for instance) the reference below.<br>
**The boosting approach to machine learning: An overview.**<br>
R. E. Schapire. *MSRI workshop on Nonlinear Estimation and Classification*, (2002).

1. [Probably Approximately Correct learning](#sec1)
2. [AdaBoost](#sec2)
3. [Implementing AdaBoost with trees](#sec3)
4. [AdaBoost in scikit-learn](#sec4)
5. [Gradient Boosting](#sec5)
6. [Examples](#sec6)
    1. [Spam or Ham?](#sec6-1)
    2. [NIST](#sec6-2)

# 1. Probably Approximately Correct learning

Probably approximately correct (PAC) learning theory helps analyze whether and under what conditions a learning algorithm will probably output an approximately correct classifier.

**"Approximately correct"**. A classifier $h(x)=y$ belonging to some family of fonctions $\mathcal{H}$ and generated by the considered learning algorithm is approximately correct if its error over the distribution $p$ of inputs (its generalization error) is bounded by some $\varepsilon$ (with $0\leq \varepsilon \leq \frac{1}{2}$), that is $\mathbb{E}_{x\sim p} \left( h\left(x\right)\neq\left(x\in c \right) \right) = \int\limits_X I_{h\left(x\right)\neq\left(x\in c\right)} \textrm{d}p(x) \leq \varepsilon$.

**"Probably"**. If, given enough data, the algorithm will output such an approximately correct classifier with probability $1-\delta$ (with $0 \leq \delta \leq \frac{1}{2}$), we call that algorithm *Probably Approximately Correct* (PAC).

In other words, an algorithm generating classifiers $h(x)=y$ is said to be a *Probably Approximately Correct* (PAC) learner (or *PAC-strong* learner) if,<br>
for all $\left\{\begin{array}{l}
p\textrm{ a distribution over } X\\
\varepsilon \in ]0;0.5[\\
\delta \in ]0;0.5[\\
c \in \mathcal{P}(X) \textrm{ a subset of X defining a classification task}
\end{array}\right.$, given enough training data,
$$\mathbb{P}\left(\int\limits_X I_{h\left(x\right)\neq\left(x\in c\right)} \textrm{d}p(x)\leq \varepsilon\right) \geq 1-\delta.$$

Knowing that a target concept $h(x)=y$ is PAC-learnable allows one to lower bound the sample size $m$ necessary to probably learn an approximately correct classifier:
$$m\geq \frac{1}{\varepsilon} \left( \log|\mathcal{H}| + \log\left(\frac{1}{\delta}\right) \right)$$

This lower bound implies that:
- As the precision requirement gets harder (as $\varepsilon$ gets closer to zero) the necessary sample size increases.
- Similarly, if the number of possible classifiers ($|\mathcal{H}|$) increases, the sample size required to disambiguate between two classifiers with good probability increases.
- Finally, as the desired probability of correctness increases (as $\delta$ gets closer to zero), the required sample size increases as well.

Want more details?<br>
**A theory of the learnable.**<br>
L.G. Valiant. *Communications of the ACM*, **27**(11):1134-1142, (1984).<br>
**The strength of weak learnability.**<br>
R.E. Schapire. *Machine Learning*, **5**(2), 197-227, (1990).

An algorithm generating classifiers $h(x)=y$ is said to be a *weak* (or *PAC-weak*) learner if, for any training set, it performs better than a random guessing on the training data.

# <a id="sec2"></a> 2. AdaBoost

AdaBoost constructs a PAC-strong classifier $f$ as a linear combination of weak classifiers $h_t(x)$:
$$f(x) = \sum\limits_{t=1}^T \alpha_t h_t(x)$$

The algorithm:
<div class="alert alert-success">
Given $\left\{\left(x_i,y_i\right)\right\}, x_i \in X, y_i \in \{-1;1\}$.<br>
Initialize weights $D_1(i) = \frac{1}{q}$<br>
For $t=1$ to $T$:
<ul>
<li> Find $h_t = \arg\min\limits_{h\in\mathcal{H}} \sum\limits_{i=1}^q D_t(i) I(y_i\neq h(x_i))$
<li> If $\epsilon_t = \sum\limits_{i=1}^q D_t(i) I(y_i\neq h_t(x_i)) \geq 1/2$ then stop
<li> Set $\alpha_t = \frac{1}{2} \log\left(\frac{1-\epsilon_t}{\epsilon_t}\right)$
<li> Update
$$D_{t+1}(i) = \frac{D_t(i) e^{-\alpha_t y_i h_t(x_i)}}{Z_t}$$
Where $Z_t$ is a normalisation factor.
</ul>

Return the classifier
$$H(x) = sign\left(\sum\limits_{t=1}^T \alpha_t h_t(x) \right)$$
</div>

Consequently, AdaBoost learns a sequence of classifiers, by performing *iterative reweighting* over the training data.

$$D_{t+1}(i) = \frac{D_t(i) e^{-\alpha_t y_i h_t(x_i)}}{Z_t}$$

- Increase the weight of incorrectly classified samples
- Decrease the weight of correctly classified samples
- Memory effect: a sample misclassified several times has a large $D(i)$
- $h_t$ focusses on samples that were misclassified by $h_0, \ldots, h_{t-1}$

Property of the training error:

$$\frac{1}{q} \sum\limits_{i=1}^q I\left(H(x_i)\neq y_i\right) \leq \prod\limits_{t=1}^T Z_t$$

- To minimize training error at each step $t$, minimize this upper bound.<br>
$\rightarrow$ This is where $\alpha_t = \frac{1}{2} \log\left(\frac{1-\epsilon_t}{\epsilon_t}\right)$ comes from.
- This is actually equivalent to maximizing a (geometrical) margin.

Many variants of AdaBoost:
- Binary classification AdaBoost.M1, AdaBoost.M2, ...
- Multiclass AdaBoost.MH,
- Regression AdaBoost.R,

And other Boosting algorithms (BrownBoost, AnyBoost...).

<div class="alert alert-success">
AdaBoost is a meta-algorithm: it "boosts" a weak classification algorithm into a committee that is a strong classifier.
<ul>
<li> AdaBoost maximizes margin
<li> Very simple to implement
<li> Can be seen as a feature selection algorithm
<li> In practice, AdaBoost often avoids overfitting.
</ul>
</div>

# <a id="sec3"></a> 3. Implementing AdaBoost with trees

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.utils import shuffle
%matplotlib inline
import matplotlib.pyplot as plt

X1, y1 = datasets.make_gaussian_quantiles(cov=2.,
                                 n_samples=300, n_features=2,
                                 n_classes=2, random_state=1)
X2, y2 = datasets.make_gaussian_quantiles(mean=(3, 3), cov=1.5,
                                 n_samples=700, n_features=2,
                                 n_classes=2, random_state=1)
X = np.concatenate((X1, X2))
y = np.concatenate((y1, - y2 + 1))
y = 2*y-1

X, y = shuffle(X, y)

Xtest,X = np.split(X,[400])
ytest,y = np.split(y,[400])

Xblue = X[y==-1]
Xred = X[y==1]
plt.figure(figsize=(7,7))
plt.scatter(Xblue[:,0],Xblue[:,1],c='b',s=20)
_=plt.scatter(Xred[:,0],Xred[:,1],c='r',s=20)

Let's grow a first tree and display it.

In [ ]:
from sklearn import tree
from os import system
from IPython.display import Image

dt1 = tree.DecisionTreeClassifier(criterion='entropy',max_depth=3)
dt1.fit(X,y)

def disp_tree(filename, treename):
    dotfile = open(filename+'.dot', 'w')
    tree.export_graphviz(treename, 
                         out_file = dotfile,
                         filled=True,
                         rounded=True,  
                         special_characters=True)
    dotfile.close()
    system("dot -Tpng "+filename+".dot -o "+filename+".png")
    return Image(filename+'.png')

disp_tree('dt1',dt1)

We would like to plot the tree's decision boundary. Let's take this occasion to plot the misclassified training points (in cyan and magenta).

In [ ]:
def plot_decision_boundary(t, X, y, fig_size=(7,7)):
    plot_step = 0.02
    x0_min, x0_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x1_min, x1_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx0, xx1 = np.meshgrid(np.arange(x0_min, x0_max, plot_step), np.arange(x1_min, x1_max, plot_step))
    yypred = t.predict(np.c_[xx0.ravel(),xx1.ravel()])
    yypred = yypred.reshape(xx0.shape)
    plt.figure(figsize=fig_size)
    plt.contourf(xx0, xx1, yypred, cmap=plt.cm.Paired)
    y_pred = t.predict(X)
    Xblue_good = X[np.equal(y,-1)*np.equal(y,y_pred)]
    Xblue_bad  = X[np.equal(y,-1)*np.not_equal(y,y_pred)]
    Xred_good  = X[np.equal(y,1)*np.equal(y,y_pred)]
    Xred_bad   = X[np.equal(y,1)*np.not_equal(y,y_pred)]
    plt.scatter(Xblue_good[:,0],Xblue_good[:,1],c='b',s=20)
    plt.scatter(Xblue_bad[:,0],Xblue_bad[:,1],c='c',marker='v',s=20)
    plt.scatter(Xred_good[:,0],Xred_good[:,1],c='r',s=20)
    plt.scatter(Xred_bad[:,0],Xred_bad[:,1],c='m',marker='v',s=20)
    plt.show()

plot_decision_boundary(dt1, X, y)
print("Training error: %g"%(1-dt1.score(X,y)))
print("Testing error:  %g"%(1-dt1.score(Xtest,ytest)))

To implement our AdaBoost algorithm, we will need a function that evaluates the majority vote of a given set of trees (a forest) and another function that plots the forest's decision boundary.

In [ ]:
def forest_predict(f, weights, X):
    N = len(f)
    votes = np.zeros((X.shape[0],N))
    for i in range(N):
        votes[:,i] = weights[i]*f[i].predict(X)
    pred = np.sum(votes,axis=1)
    return np.sign(pred)

def plot_decision_boundary_forest(f, weights, X, y, fig_size=(7,7)):
    plot_step = 0.02
    x0_min, x0_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x1_min, x1_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx0, xx1 = np.meshgrid(np.arange(x0_min, x0_max, plot_step), np.arange(x1_min, x1_max, plot_step))
    yypred = forest_predict(f, weights, np.c_[xx0.ravel(),xx1.ravel()])
    yypred = yypred.reshape(xx0.shape)
    plt.figure(figsize=fig_size)
    plt.contourf(xx0, xx1, yypred, cmap=plt.cm.Paired)
    y_pred = forest_predict(f, weights, X)
    Xblue_good = X[np.equal(y,-1)*np.equal(y,y_pred)]
    Xblue_bad  = X[np.equal(y,-1)*np.not_equal(y,y_pred)]
    Xred_good  = X[np.equal(y,1)*np.equal(y,y_pred)]
    Xred_bad   = X[np.equal(y,1)*np.not_equal(y,y_pred)]
    plt.scatter(Xblue_good[:,0],Xblue_good[:,1],c='b',s=20)
    plt.scatter(Xblue_bad[:,0],Xblue_bad[:,1],c='c',marker='v',s=20)
    plt.scatter(Xred_good[:,0],Xred_good[:,1],c='r',s=20)
    plt.scatter(Xred_bad[:,0],Xred_bad[:,1],c='m',marker='v',s=20)
    plt.show()

forest = [dt1]
plot_decision_boundary_forest(forest,np.ones(1),X,y)

Now we can implement an AdaBoost algorithm with trees and visualize its decision boundary.<br>
<div class="alert alert-warning">
    
**Exercice**:<br>
Using the elements introduced above, implement AdaBoost on a forest containing 100 trees, trained on the current dataset.<br>
Display how the decision boundary evolves.<br>
Plot on the same graph the evolution of the training and testing errors.
</div>

In [ ]:
# %load solutions/code1.py
### WRITE YOUR CODE HERE
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).


<div class="alert alert-warning">
    
**Question**:
Is there a tendency to overfit?
</div>

# <a id="sec4"></a> 4. AdaBoost in scikit-learn

Scikit-learn provides an [AdaBoost](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier) meta-algorithm.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

boosted_forest = AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy',max_depth=3), n_estimators=100)
boosted_forest.fit(X,y)

plot_decision_boundary(boosted_forest,X,y)
print("Training score:", boosted_forest.score(X,y))
print("Testing score: ", boosted_forest.score(Xtest,ytest))

# <a id="sec5"></a> 5. Gradient Boosting

AdaBoost incrementally builds a committee of classifiers (or regressors) where each one tries to compensate the weaknesses of all the previous ones. This descriptions makes it sound a lot like gradient descent.

Let's write $f_k = \{h_0,\ldots,h_k\}$ the committee predictor obtained at step $k$. $f_k$ is a point in $\mathcal{H}$ and our goal is to find the function (the point) in $\mathcal{H}$ that minimizes a certain loss function $L(f), \ f\in\mathcal{H}$. Therefore, $h_{k+1}$ should point in the opposite direction of the gradient of $L$ with respect to functions $f$.

This is precisely the idea of Gradient Boosting, that extends traditionnal Boosting to any differentiable loss function. Let's write this formally.

Suppose we have a loss function $L(f(x),y)$ that quantifies how bad a predictor $f$ is, given the true data pair $(x,y)$. This could be a squared error, a cross-entropy, or any other loss function that compares $f(x)$ to the true $(x,y)$ data.

The goal of a learning algorithm is to find
$$f^* = \arg\min_{f\in\mathcal{H}} \mathbb{E}_{(x,y)\sim p(x,y)} \left( L(f(x),y) \right)$$

To simplify the notation, we will write $\mathbb{E}_{x,y}$ for $\mathbb{E}_{(x,y)\sim p(x,y)}$.

So if we assume an current function $f_{k-1}$, one can perform gradient descent by writing 
$$f_{k} = f_{k-1} + \alpha_{k} h_{k}$$
Where
$$h_{k} = \nabla_f \left[\mathbb{E}_{x,y} \left( L(f_{k-1}(x),y) \right)\right] = \mathbb{E}_{x,y} \left[ \nabla_f L(f_{k-1}(x),y) \right]$$
And $\alpha_{k}$ is found by line search
$$\alpha_{k} = \arg\min_\alpha \left[ \mathbb{E}_{x,y} \left[ L(f_{k-1}(x) + \alpha h_{k}(x),y) \right] \right]$$

Of course, the true value of $\mathbb{E}_{x,y} \left[ \nabla_f L(f_k(x),y) \right]$ is not accessible since it would require an infinite amount of data. But we can still approximate it using the training set:
$$h_{k} = \sum_{i} \nabla_f L(f_{k-1}(x_i),y_i)$$
And similarly
$$\alpha_{k} = \arg\min_\alpha \left[ \sum_{i} L(f_{k-1}(x_i) + \alpha h_{k}(x_i),y) \right]$$

Taking the gradient with respect to $f$ functions in $\mathcal{H}$ is quite an abstract operation, so how do we choose the descent direction in practice? The key remark here is to notice that, for a given function $f$, although $f$ lives in a possibly infinite-dimensional function space, $f(x_i)$ lives in $\mathbb{R}$ and is a descriptor of $f$ (around $x_i$). Thus, the function that would fit the training set $\left\{ \left(x_i, \left[\frac{\partial L(f(x_i), y_i)}{\partial f(x_i)}\right]_{f = f_{k-1}} \right) \right\}$ is an approximate descent direction.

The initial function $f_0$ is chosen to be a constant function, so:
$$f_0 = \arg\min_\gamma \sum_i L(\gamma, y_i)$$

So the algorithm can be written:
<div class="alert alert-success">
$f_0 = \arg\min_\gamma \sum_i L(\gamma, y_i)$<br>
For $k=1$ to $K$
<ul>
<li> Compute the pseudo-residuals
$$r_i = \left[ \frac{\partial L(f(x_i), y)}{\partial f(x_i)} \right]_{f = f_{k-1}}$$
<li> Train $h_k$ to fit the dataset $\left(x_i, r_i\right)$
<li> Find $\alpha_k$ though line search
$$\alpha_{k} = \arg\min_\alpha \left[ \sum_i L\left(f_{k-1}(x_i) + \alpha h_{k}(x_i),y_i\right) \right]$$
<li> Update the model
$$f_k = f_{k-1} + \alpha_k h_k$$
</ul>
Return $f_K$
</div>

AdaBoost is actually a Gradient Boosting procedure.

When $\mathcal{H}$ is the set of regression or classification trees, Gradient Boosting is called **Gradient Tree Boosting**. Several extensions (regularization, local step-sizes, etc.) are possible in that case.

The most well-known Gradient Boosting library is called [XGBoost](https://github.com/dmlc/xgboost). It is efficient and quite flexible. It has be used to win several Data Science competitions. But Scikit-Learn also provides a decent implementation that we will use below.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gtb = GradientBoostingClassifier(n_estimators=100)
gtb.fit(X,y)

plot_decision_boundary(gtb,X,y)
print("Training score:", gtb.score(X,y))
print("Testing score: ", gtb.score(Xtest,ytest))

# <a id="sec6"></a> 6. Examples

If you have done the previous notebooks, you are used to these two examples now.

## <a id="sec6-1"></a>6.1 Spam or ham?

In [ ]:
from sys import path
path.append('../2 - Text data preprocessing')
import load_spam
spam_data = load_spam.spam_data_loader()
spam_data.load_data()

Xtrain, ytrain, Xtest, ytest = spam_data.split(2000)

In [ ]:
spam_ada = AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy',max_depth=3), n_estimators=100)
spam_ada.fit(Xtrain,ytrain)
print("score:", spam_ada.score(Xtest,ytest))

In [ ]:
spam_gtb = GradientBoostingClassifier(n_estimators=100)
spam_gtb.fit(Xtrain,ytrain)
print("score:", spam_gtb.score(Xtest,ytest))

In [ ]:
Xtrain, ytrain, Xtest, ytest = spam_data.split(2000, feat='wordcount')

In [ ]:
spam_ada = AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy',max_depth=3), n_estimators=100)
spam_ada.fit(Xtrain,ytrain)
print("score:", spam_ada.score(Xtest,ytest))

In [ ]:
spam_gtb = GradientBoostingClassifier(n_estimators=100)
spam_gtb.fit(Xtrain,ytrain)
print("score:", spam_gtb.score(Xtest,ytest))

## <a id="sec6-2"></a> 6.2 NIST

In [ ]:
from sklearn import datasets
%matplotlib inline
import matplotlib.pyplot as plt

digits = datasets.load_digits()
#print(digits.data.shape)
#print(digits.images.shape)
#print(digits.target.shape)
#print(digits.target_names)

X = digits.data
y = digits.target
Xtrain,Xtest = np.split(X,[1000])
ytrain,ytest = np.split(y,[1000])

#print(digits.DESCR)

In [ ]:
from sklearn.utils import shuffle

def shuffle_and_split(X,y,n):
    X0,y0 = shuffle(X,y)
    Xtrain,Xtest = np.split(X0,[n])
    ytrain,ytest = np.split(y0,[n])
    return Xtrain, ytrain, Xtest, ytest

In [ ]:
print(Xtrain.shape)
print(ytrain.shape)
digits_ada = AdaBoostClassifier(tree.DecisionTreeClassifier(criterion='entropy',max_depth=3), n_estimators=100)
digits_ada.fit(Xtrain,ytrain)
prediction = digits_ada.predict(Xtest)
#print("Training error:", np.sum(np.not_equal(prediction,ytest))/len(ytest))
print("Generalization error:", np.sum(np.not_equal(prediction,ytest))/len(ytest) )
print("Generalization score:", digits_ada.score(Xtest,ytest))

In [ ]:
digits_gtb = GradientBoostingClassifier(n_estimators=100)
digits_gtb.fit(Xtrain,ytrain)
prediction = digits_ada.predict(Xtest)
#print("Training error:", np.sum(np.not_equal(prediction,ytest))/len(ytest))
print("Generalization error:", np.sum(np.not_equal(prediction,ytest))/len(ytest) )
print("Generalization score:", digits_gtb.score(Xtest,ytest))